In [1]:
import pandas as pd
import numpy as np
import random
import math
from tqdm import tqdm

In [3]:
def ci_summary(my_list, percent=True, Label='Mean'):
  # Calculate mean
  mean_val = np.mean(my_list)
  # Calculate 2.5 percentile
  percentile_025 = np.percentile(my_list, 2.5)
  # Calculate 97.5 percentile
  percentile_975 = np.percentile(my_list, 97.5)
  if percent==True:
    print(f"\n{Label}: {mean_val*100:.2f}%, (95% SI: {percentile_025*100:.2f}%, {percentile_975*100:.2f}%)")
  else:
    print(f"\n{Label}: {mean_val:.2f}, (95% SI: {percentile_025:.2f}, {percentile_975:.2f})")

In [4]:
def beta_parameters(mean, stddev):
    # make sure mean and stddev are in valid range
    assert 0 < mean < 1, "Mean should be in (0, 1) range"
    assert 0 < stddev < np.sqrt(mean * (1 - mean)), "Standard deviation should be in (0, sqrt(mean * (1 - mean))) range"
    
    # convert stddev to variance
    variance = stddev ** 2
    
    # common part of both alpha and beta
    common = mean * (1 - mean) / variance - 1
    
    # calculate alpha and beta
    alpha = mean * common
    beta = (1 - mean) * common
    
    return alpha, beta

In [5]:
asd_prev=pd.read_csv('ASD Prevalence.csv')
asd_prev['Age_group']=18
pop = pd.read_csv('Population Data.csv')
mortality_rate=pd.read_csv('Mortality Rates for General Population.csv')
hr = pd.read_csv('ASD Mortality HR.csv')

In [6]:
# Create a boolean mask for rows that contain the text
import re

# Define the age groups to match
age_groups = ['0 to 4', '5 to 9', '10 to 14', '90 to 94', '95 to 99', '100 and over']

# Create a regex pattern that matches any of the age groups as a whole word (not part of another string)
# The \b denotes a word boundary in regex, ensuring we're matching the entire string and not a part of it.
pattern = r'\b(?:' + '|'.join(age_groups) + r')\b'

# Apply a function to each cell in the DataFrame. The function checks if the cell's value (converted to a string and stripped of leading/trailing whitespace)
# matches the regex pattern exactly. This returns a DataFrame of the same shape as 'pop' with True for matches and False for non-matches.
# Then, .any(axis=1) checks if any cells in each row are True (i.e., if any cells in the row match one of the age groups).
# This results in a Series where each element corresponds to a row of 'pop', and the element is True if any cell in that row matches one of the age groups.
mask = pop.apply(lambda row: row.astype(str).apply(lambda x: bool(re.fullmatch(pattern, x.strip()))).any(), axis=1)

# Filter the DataFrame to exclude the rows with the specified text
pop = pop[~mask].copy()
pop.reset_index(inplace=True, drop=True)

# Replace "15 to 19" with "18 to 19" in the "Age_group" column
pop['Age_group'] = np.where(pop["Age_group"]=="15 to 19", "18 to 19", pop["Age_group"])

# Convert "object_column" to integer
pop['Persons'] = pop['Persons'].astype(int)

# Conditionally divide "person" column values by 5 and multiply by 2
pop['Persons'] = np.where(pop["Age_group"]=="18 to 19", (pop["Persons"]/5)*2, pop["Persons"])


# Create a new DataFrame to store the new data
dfs = []

# Iterate through the original DataFrame
for index, row in pop.iterrows():
    # Get the start and end ages for each age group
    start_age, end_age = [int(age) for age in row['Age_group'].split(' to ')]
    
    # Create new rows for each individual age within the age group
    for age in range(start_age, end_age + 1):
        new_row = row.copy()
        new_row['Age_group'] = age
        new_row['Persons'] = new_row['Persons'] / 2 if new_row['Age_group'] == 18 or new_row['Age_group'] == 19 else new_row['Persons'] / 5

        new_row['Persons']=round(new_row['Persons'],0)
        dfs.append(pd.DataFrame(new_row).T)

        # Concatenate the list of DataFrames into a single DataFrame
new_data = pd.concat(dfs, ignore_index=True)
new_data['Persons']=new_data['Persons'].astype(int)
pop=new_data.copy()

In [7]:
# Create a boolean mask for rows that contain the text
mask = mortality_rate.apply(lambda row: any(row.astype(str).str.contains('0 to 1|1 to 4|5 to 9|10 to 14|90+')), axis=1)

# Filter the DataFrame to exclude the rows with the text
mortality_rate = mortality_rate[~mask].copy()
mortality_rate.reset_index(inplace=True, drop=True)


# Replace "15 to 19" with "18 to 19" in the "Age_group" column
mortality_rate['Age_group'] = np.where(mortality_rate["Age_group"]=="15 to 19", "18 to 19", mortality_rate["Age_group"])

# Convert "object_column" to integer
mortality_rate['Rate'] = mortality_rate['Rate'].astype(float)
mortality_rate = mortality_rate[mortality_rate['Year'] == 2019]
mortality_rate = mortality_rate.drop('Year', axis=1)
mortality_rate = mortality_rate.rename(columns={'Rate': 'Mortality Rate'})

# Create a new DataFrame to store the new data
dfs = []

# Iterate through the original DataFrame
for index, row in mortality_rate.iterrows():
    # Get the start and end ages for each age group
    start_age, end_age = [int(age) for age in row['Age_group'].split(' to ')]
    
    # Create new rows for each individual age within the age group
    for age in range(start_age, end_age + 1):
        new_row = row.copy()
        new_row['Age_group'] = age
        dfs.append(pd.DataFrame(new_row).T)
# Concatenate the list of DataFrames into a single DataFrame
new_data = pd.concat(dfs, ignore_index=True)
mortality_rate=new_data.copy()

In [8]:
data = pd.DataFrame()
data = pd.merge(left=pop,right=mortality_rate, how='left', on=['Geography','Sex','Age_group'])
data = pd.merge(left=data,right=asd_prev, how='left', on=['Geography','Sex', 'Age_group'])
data = pd.merge(left=data,right=hr, how='left', on=['Sex'])

In [9]:
data = data.rename(columns={'Mortality Rate':'mortality_rate', 'rate':'hazard_ratio'})
data['Persons']=data['Persons'].astype(int)
data['hazard_ratio']=np.exp(data['hazard_ratio'])


data["mortality_rate"]=data["mortality_rate"].astype(float)
data["Persons"]=data["Persons"].astype(float)
data["asd_prevalence"]=data["asd_prevalence"].astype(float)

data['pop_survival']=1-((data['Persons']*((data['mortality_rate']/1000)))/data['Persons'])
data = data.rename(columns={'Persons':'population', 'Age_group': 'age', 'Geography':'province', 'Sex':'sex'})


In [10]:
data[(data['province'] == 'Alberta') & (data['sex'] == 'Female')]

# province = The name of Canadain province
# sex = sex male/females
# Age = Age of group increasing by 1 year increments
# population = Population size for the respective province, sex, and age group. This data was obtained from Statistics canada
# mortality_rate = Mortality rate per 1000 individuals from the general population
# asd_prevalence = asd prevalence from 2019 CHSCY for those 1-17 years of age.
# asd_prevalence_se = standard error for asd prevalence from 2019 CHSCY for those 1-17 years of age.
# hazard_ratio = hazard ratio for death for the respective sex. We assumed constant hazard ratio for all ages and all provinces.
# se = standard error for the hazard ratio for death in the respective sex. We assumed constant hazard ratio for all ages and all provinces.
# pop_survival = probability of survival for the respective province, sex, and age group. This variable was derived from mortality_rate.


,province,sex,age,population,mortality_rate,asd_prevalence,asd_prevalence_se,hazard_ratio,se,pop_survival
1224,Alberta,Female,18,24840.0,0.2,0.009583,0.003102,3.119897,0.145,0.9998
1225,Alberta,Female,19,24840.0,0.2,NaN,NaN,3.119897,0.145,0.9998
1226,Alberta,Female,20,26493.0,0.5,NaN,NaN,3.119897,0.145,0.9995
1227,Alberta,Female,21,26493.0,0.5,NaN,NaN,3.119897,0.145,0.9995
1228,Alberta,Female,22,26493.0,0.5,NaN,NaN,3.119897,0.145,0.9995
...,...,...,...,...,...,...,...,...,...,...
1291,Alberta,Female,85,5109.0,72.8,NaN,NaN,3.119897,0.145,0.9272
1292,Alberta,Female,86,5109.0,72.8,NaN,NaN,3.119897,0.145,0.9272
1293,Alberta,Female,87,5109.0,72.8,NaN,NaN,3.119897,0.145,0.9272
1294,Alberta,Female,88,5109.0,72.8,NaN,NaN,3.119897,0.145,0.9272


### Step 6: Conduct analysis to estimate prevalence
This code below is manipulating a dataset based on ASD (Autism Spectrum Disorder). It's iterating through groups of data by province and sex, calculating various statistics such as ASD prevalence, mortality rate, number of ASD cases, ASD survival rate, rho_adj, and gamma_adj, and appending these values to respective lists.

**Methods**

In our research, we employed a simulation-based methodology to estimate the prevalence of Autism Spectrum Disorder (ASD) across various Canadian provinces. This involved considering a range of factors such as age, sex, and overall population. The process hinged on a Monte Carlo simulation, a recognized technique in computational statistics that allows for the generation of numerous potential outcomes and their probabilities through random sampling.

Initially, we categorized our dataset into distinct groups based on the province and sex, with each group encapsulating data associated with age, population, and several ASD-related parameters.

However, we were unable to derive estimates for ASD prevalence and standard errors for Yukon, Northwest Territories, Nunavut, and the female population in Prince Edward Island using the Canadian Health Survey on Children and Youth (CHSCY). To manage this data limitation, we made two informed assumptions. Firstly, we assumed that the ASD prevalence in these regions would align with the national sex-specific average. Secondly, we proposed that the standard error in these regions would equate to 50% of the ASD prevalence, signifying a coefficient of variation of 50%.

Within each simulation run, the hazard ratio for death was recalculated by selecting a value from a normal distribution defined by a specific mean and standard error. This technique facilitated the simulation of a plausible level of random variation.

For every province-sex group, the initial calculation of ASD prevalence was based on a presumed normal distribution centered around the given prevalence value and its standard error. 

Following this, within each age category of a group, we calculated several variables. These included the ASD mortality rate, calculated as the product of the group's hazard ratio for death and the survival rate of the general population. The number of ASD cases was also calculated, using the group's population and the ASD prevalence. An adjusted ASD prevalence was computed to reflect the probable effect of the ASD survival rate on the prevalence. Lastly, the adjusted number of ASD cases in the population was estimated by multiplying the group's population by the adjusted ASD prevalence.

This procedure was repeated across all age groups within each province-sex grouping. The data from each simulation run, including the calculated values and the province, sex, age, and population data, was then aggregated. 

This approach was conducted over a pre-set number of simulations, each iteration generating a unique data scenario, thereby constructing a wide array of plausible outcomes. This extensive collection of scenarios enabled a comprehensive understanding of the variability and potential range of ASD prevalence across Canadian provinces.

This simulation-based approach provided a robust mechanism for estimating ASD prevalence, taking into account inherent uncertainties in population-based studies and potential random variation in key parameters such as the hazard ratio for death and ASD prevalence. Additionally, it addressed data limitations by making evidence-based assumptions where required.

In [11]:
def run_simulation(data, hr, beta_parameters, num_simulations=1000):
    # Group the data by 'province' and 'sex' columns
    grouped = {province: data for province, data in data.groupby(['province','sex'])}

    sim_data = []

    # Initialize the tqdm progress bar
    for num_sim in tqdm(range(num_simulations), desc="Running simulations"):
        hr['current_simulated_hr'] = np.exp(np.random.normal(loc=hr['rate'], scale=hr['se']))

        # Initialize empty lists to store data
        province, sex, ages, pops, ASD_prev_3_17, N_ASD, HR, asd_survival, rho_adj, gamma_adj = ([] for _ in range(10))

        # Loop through each key in the dictionary
        for key in grouped:
            # Get ASD prevalence for the current group
            asd_prevalence_prev = grouped[key]['asd_prevalence'].values[0]
            se_ASD_prev=grouped[key]['asd_prevalence_se'].values[0]

            alpha, beta = beta_parameters(asd_prevalence_prev, se_ASD_prev)
            asd_prevalence_prev = np.random.beta(alpha, beta, 1)[0]
            #asd_prevalence_prev = np.random.normal(loc=asd_prevalence_prev, scale=se_ASD_prev)

            # Store the first ASD prevalence for the current group
            first_asd_prevalence = asd_prevalence_prev

            # Get ASD mortality rate for the current group
            mortality_rate = grouped[key]['mortality_rate'].values
            exponentiated_HR = hr[hr['Sex'] == key[1]]['current_simulated_hr'].values[0]
            
            # Get population for the current group
            population = grouped[key]['population'].values
            # Get population survival rate for the current group
            pop_survival = grouped[key]['pop_survival'].values
            # Get age for the current group
            age = grouped[key]['age'].values

            # Loop through each record in the current group
            for i, (age_val, pop_val, pop_survival_val) in enumerate(zip(age, population, pop_survival)):
                asd_mortality_rate = exponentiated_HR * pop_survival_val
                asd_survival_temp = 1 - (asd_mortality_rate / 1000)
                N_ASD_temp = pop_val * asd_prevalence_prev
                rho_adj_temp = asd_prevalence_prev * asd_survival_temp
                gamma_adj_temp = pop_val * rho_adj_temp

                # Append province, sex and age to their respective lists
                province.append(key[0])
                sex.append(key[1])
                ages.append(age_val)

                # Append population and ASD prevalence for ages 3-17 to their respective lists
                pops.append(pop_val)
                ASD_prev_3_17.append(first_asd_prevalence)
                
                # Calculate number of ASD cases and append to the list
                N_ASD.append(N_ASD_temp)
                # Calculate ASD survival rate and append to the list
                asd_survival.append(asd_survival_temp)
                # Calculate rho_adj (adjusted ASD prevalence) and append to the list
                rho_adj.append(rho_adj_temp)
                # Calculate gamma_adj (adjusted ASD cases in the population) and append to the list
                gamma_adj.append(gamma_adj_temp)
                HR.append(exponentiated_HR)
                # Update the ASD prevalence (from current ASD province, sex, age group) for the next iteration
                asd_prevalence_prev = rho_adj_temp if age_val != 89 else 0

        mydata = {'province': province, 'age': ages, 'sex': sex, 'population': pops,
                  'ASD_prev_3_17': ASD_prev_3_17, 'N_ASD': N_ASD, 'Hazard_Ratio': HR,
                  'asd_survival': asd_survival, 'rho_adj': rho_adj, 
                  'gamma_adj': gamma_adj}

        data2 = pd.DataFrame(mydata)
        sim_data.append(data2)

    return sim_data

# Use the function
if __name__ == "__main__":
    sim_data = run_simulation(data, hr, beta_parameters)

Running simulations: 100%|██████████| 1000/1000 [00:16<00:00, 61.01it/s]


### Functions to aggregate simulated data

#### By a particular group

In [85]:
def get_stats_grouped(df, variables):
    # Define bins and labels
    bins = [18, 20, 25, 30,35,40,45,50,55,60,65,70,75,80,85, np.inf]
    labels = ['18-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54',
              '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85+']

    # Add "Simulation Number" to each dataframe in the list before concatenating
    for i in tqdm(range(len(sim_data)), desc="Processing simulation data"):
        sim_data[i]['Simulation Number'] = i

    aggregated_sim_data = pd.concat(df)

    # Create a new column 'age_group' based on 'age' column
    aggregated_sim_data['age_group'] = pd.cut(aggregated_sim_data['age'], bins=bins, labels=labels, right=False)
    aggregated_sim_data=aggregated_sim_data.reset_index(drop=True)
    aggregated_sim_data = aggregated_sim_data[variables+['population','gamma_adj']+["Simulation Number"]].groupby(variables+["Simulation Number"]).sum().reset_index()
    aggregated_sim_data['rho_adj']=aggregated_sim_data['gamma_adj']/aggregated_sim_data['population']

    # Apply the aggregation functions to each group
    descriptive_stats = df_grouped.groupby(variables).agg(agg_funcs).reset_index()
    # Flatten the MultiIndex columns
    descriptive_stats.columns = ['_'.join(col).rstrip('_') for col in descriptive_stats.columns.values]

    # Rename the columns
    descriptive_stats.rename(columns={
        'rho_adj_mean': 'Mean',
        'rho_adj_<lambda_0>': '2.5th SI',
        'rho_adj_<lambda_1>': '97.5th SI',
        'rho_adj_min': 'Minimum',
        'rho_adj_max': 'Maximum',
    }, inplace=True)
    return descriptive_stats

#### Nationally

In [86]:
def get_stats(df, column):
    """
    Given a dataframe and column name, this function will return
    the mean, 2.5th percentile, 97.5th percentile, min, and max of the column.
    These statistics will be returned as percentages.
    """
    mean = df[column].mean() * 100
    percentile_2_5 = np.percentile(df[column], 2.5) * 100
    percentile_97_5 = np.percentile(df[column], 97.5) * 100
    min_val = df[column].min() * 100
    max_val = df[column].max() * 100

    return {
        "mean": f'{mean:.2f}%',
        "2.5th SI": f'{percentile_2_5:.2f}%',
        "97.5th SI": f'{percentile_97_5:.2f}%',
        "min": f'{min_val:.2f}%',
        "max": f'{max_val:.2f}%',
    }

## Results

### 1. Results aggregated dataset by Province, Sex, Age Group, and Simulation Number

In [87]:
variables=['province', 'sex', 'age_group']
get_stats_grouped(sim_data, variables=variables)

Processing simulation data: 100%|██████████| 1000/1000 [00:00<00:00, 15211.25it/s]


,province,sex,age_group,Mean,2.5th SI,97.5th SI,Minimum,Maximum
0,Alberta,Female,18-19,0.009513,0.004299,0.016500,0.002081,0.022141
1,Alberta,Female,20-24,0.009409,0.004257,0.016305,0.002059,0.021855
2,Alberta,Female,25-29,0.009262,0.004195,0.016031,0.002027,0.021452
3,Alberta,Female,30-34,0.009117,0.004124,0.015773,0.001995,0.021057
4,Alberta,Female,35-39,0.008975,0.004055,0.015533,0.001964,0.020669
...,...,...,...,...,...,...,...,...
385,Yukon,Male,65-69,0.026906,0.007212,0.059948,0.003248,0.087643
386,Yukon,Male,70-74,0.026495,0.007098,0.059001,0.003206,0.086473
387,Yukon,Male,75-79,0.026097,0.006987,0.058062,0.003165,0.085338
388,Yukon,Male,80-84,0.025710,0.006880,0.057082,0.003125,0.084235


### 2. Results by Province and Sex

In [88]:
variables=['province', 'sex']
get_stats_grouped(sim_data, variables=variables)

Processing simulation data: 100%|██████████| 1000/1000 [00:00<00:00, 13735.29it/s]


,province,sex,Mean,2.5th SI,97.5th SI,Minimum,Maximum
0,Alberta,Female,0.008581,0.003801,0.015294,0.001683,0.022141
1,Alberta,Male,0.027218,0.018243,0.037844,0.011643,0.047150
2,British Columbia,Female,0.007348,0.003762,0.012687,0.002210,0.018189
3,British Columbia,Male,0.031323,0.021822,0.042622,0.015244,0.061046
4,Manitoba,Female,0.003854,0.000939,0.009150,0.000469,0.013792
5,Manitoba,Male,0.026334,0.014234,0.040979,0.007085,0.051404
6,New Brunswick,Female,0.009867,0.002870,0.021021,0.001268,0.041550
7,New Brunswick,Male,0.062261,0.039070,0.089851,0.030252,0.114707
8,Newfoundland and Labrador,Female,0.003215,0.000860,0.007213,0.000207,0.013427
9,Newfoundland and Labrador,Male,0.035827,0.019313,0.057340,0.011526,0.088232


### 3. Results by Province

In [89]:
variables=['province']
get_stats_grouped(sim_data, variables=variables)

Processing simulation data: 100%|██████████| 1000/1000 [00:00<00:00, 12147.97it/s]


,province,Mean,2.5th SI,97.5th SI,Minimum,Maximum
0,Alberta,0.017899,0.004380,0.035924,0.001683,0.047150
1,British Columbia,0.019335,0.004146,0.040670,0.002210,0.061046
2,Manitoba,0.015094,0.001204,0.038763,0.000469,0.051404
3,New Brunswick,0.036064,0.003534,0.084845,0.001268,0.114707
4,Newfoundland and Labrador,0.019521,0.001093,0.052964,0.000207,0.088232
5,Northwest Territories,0.017542,0.002407,0.056112,0.000527,0.098981
6,Nova Scotia,0.013371,0.001338,0.031928,0.000286,0.048429
7,Nunavut,0.017553,0.002390,0.055905,0.000733,0.107230
8,Ontario,0.018761,0.005881,0.034736,0.003824,0.041200
9,Prince Edward Island,0.031773,0.002317,0.074187,0.000577,0.099050


### 4. Results aggregated dataset Nationally

In [90]:
variables=["Simulation Number"]

# Add "Simulation Number" to each dataframe in the list before concatenating
for i in tqdm(range(len(sim_data)), desc="Processing simulation data"):
    sim_data[i]['Simulation Number'] = i

aggregated_sim_data = pd.concat(sim_data)
aggregated_sim_data=aggregated_sim_data.reset_index(drop=True)
aggregated_sim_data = aggregated_sim_data[variables+['population','gamma_adj']].groupby(variables).sum().reset_index()
aggregated_sim_data['rho_adj']=aggregated_sim_data['gamma_adj']/aggregated_sim_data['population']

rho_adj_stats = get_stats(aggregated_sim_data, 'rho_adj')
rho_adj_stats

Processing simulation data: 100%|██████████| 1000/1000 [00:00<00:00, 14323.98it/s]


{'mean': '1.78%',
 '2.5th SI': '1.63%',
 '97.5th SI': '1.95%',
 'min': '1.54%',
 'max': '2.11%'}

### 5. Results aggregated dataset nationally by sex

In [91]:
variables=['sex']
get_stats_grouped(sim_data, variables=variables)

Processing simulation data: 100%|██████████| 1000/1000 [00:00<00:00, 15660.20it/s]


,sex,Mean,2.5th SI,97.5th SI,Minimum,Maximum
0,Female,0.00674,0.001392,0.015259,0.000207,0.041550
1,Male,0.03123,0.006982,0.071961,0.001201,0.114707


### 6. Results aggregated dataset nationally by age group

In [92]:
variables=['age_group']
get_stats_grouped(sim_data, variables=variables)

Processing simulation data: 100%|██████████| 1000/1000 [00:00<00:00, 13188.10it/s]


,age_group,Mean,2.5th SI,97.5th SI,Minimum,Maximum
0,18-19,0.021044,0.002013,0.072172,0.000254,0.114707
1,20-24,0.020813,0.001993,0.071316,0.000251,0.113359
2,25-29,0.020488,0.001962,0.070153,0.000247,0.111459
3,30-34,0.020168,0.001929,0.069011,0.000244,0.109590
4,35-39,0.019853,0.001898,0.067824,0.000240,0.107753
5,40-44,0.019543,0.001868,0.066799,0.000237,0.106035
6,45-49,0.019238,0.001839,0.065720,0.000233,0.104485
7,50-54,0.018938,0.001810,0.064705,0.000230,0.102958
8,55-59,0.018644,0.001783,0.063703,0.000226,0.101456
9,60-64,0.018355,0.001755,0.062680,0.000223,0.099981


In [93]:
### 5. Results aggregated dataset nationally by sex and age group

In [94]:
variables=['sex', 'age_group']
get_stats_grouped(sim_data, variables=variables)

Processing simulation data: 100%|██████████| 1000/1000 [00:00<00:00, 15666.81it/s]


,sex,age_group,Mean,2.5th SI,97.5th SI,Minimum,Maximum
0,Female,18-19,0.007470,0.001559,0.016755,0.000254,0.041550
1,Female,20-24,0.007388,0.001541,0.016576,0.000251,0.041095
2,Female,25-29,0.007273,0.001520,0.016323,0.000247,0.040453
3,Female,30-34,0.007160,0.001492,0.016080,0.000244,0.039822
4,Female,35-39,0.007048,0.001468,0.015853,0.000240,0.039200
5,Female,40-44,0.006938,0.001444,0.015606,0.000237,0.038588
6,Female,45-49,0.006830,0.001421,0.015327,0.000233,0.037986
7,Female,50-54,0.006724,0.001402,0.015077,0.000230,0.037394
8,Female,55-59,0.006619,0.001376,0.014858,0.000226,0.036812
9,Female,60-64,0.006517,0.001357,0.014630,0.000223,0.036240
